### Setup

In [1]:
# extract the api key from the secrets file
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Replace 'API_KEY' with the name of the variable containing the API key in your .env file
OPEN_AI_KEY = os.getenv('OPEN_AI_KEY')


In [2]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import Agent

In [20]:
import json
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
from bard import ask_internet
from slack import send_message

def read_slack(input):
    """
    Reads all the messages from a slack export file
    """    
    with open('data/slack/hackathon_channel.csv', 'r') as f:
        csv_file = f.read()
    
    return csv_file

def read_bio(name):
    """
    Reads the bio from a linkedin profile
    """    
    with open('data/bios/all.json', 'r') as f:
        json_file = f.read()
    
    # extract only the bio of 'name'
    json_file = json.loads(json_file)
    
    return json_file

# def read_linkedin_bio(input):
#     """
#     Reads the bio from a linkedin profile
#     """    
#     with open('data/linkedin/hamudi.json', 'r') as f:
#         json_file = f.read()
    
#     return json_file

def analyse(task):
    """
    Analyse the input using the OpenAI API
    """
    prompt = PromptTemplate(
        input_variables=['task'],
        template="{task}"
    )

    analyzer_llm = OpenAI(
        openai_api_key=OPEN_AI_KEY, 
        model_name='gpt-3.5-turbo-16k'
    )

    chain = LLMChain(
        llm=analyzer_llm,
        prompt=prompt,
    )
    
    chain.run({
        'task': task
    })

llm = ChatOpenAI(openai_api_key=OPEN_AI_KEY, model_name="gpt-3.5-turbo-16k")

tools = [
    Tool.from_function(
        func=read_slack,
        name="read_slack",
        description="Useful to extract all slack messages from all people"
    ),
    Tool.from_function(
        func=analyse,
        name="analyse",
        description="Useful to run general analysis of a given text"
    ),
    # Tool.from_function(
    #     func=read_linkedin_bio,
    #     name="read_bio",
    #     description="Useful to extract the bio of a person given their name"
    # ),
    Tool.from_function(
        func=ask_internet,
        name="write summary",
        description="Useful to write summary about how to approach people"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [28]:
agent.run(
    """I'm meeting with Harrison later today to discuss his new open source framework. 
    Give me a summary of things he talked about recently."""
)



> Entering new AgentExecutor chain...
I should read the slack messages to find out Harrison's main interests.
Action: read_slack
Action Input: All slack messages
Observation: user,text,ts
Rocky Yu,Dinner will be ready in 20min or so ,2023-07-22 17:32:58
Rocky Yu,Picking up food now ,2023-07-22 17:32:42
Eduardo Reis,"Hey everyone, letting you know that few emails from <http://llama-api.com|llama-api.com> are going to spam for some people, if you didn’t get the invite, please check you spam box.",2023-07-22 16:53:27
Robert Nowell,"Hi folks, anyone want an extra pair of hands to test / improve prompts? Feel free to message",2023-07-22 16:15:22
Ron Bodkin,"Hi all - I wanted to share a link our new open source framework for building with collaborating agents, Council: <https://github.com/chain-ml/council> - I'd love to get feedback and be happy to answer questions if you try it out.",2023-07-22 16:12:22
Eduardo Reis,"We are giving credits for teams running Llama API - the first implementa

"Harrison's main interests are the open source framework Council, LangSmith, and OpenAI function calling and custom prompts."

In [22]:
agent.run(
    """I'm looking for a list of people from slack that work with robots from different perspectives. 
    How do I approach them individually based on their backgrounds, personalities, aspirations, and beliefs?"""
)



> Entering new AgentExecutor chain...
To find a list of people from slack who work with robots, I can use the "read_slack" tool. Then, I can use the "write summary" tool to write a paragraph on how to approach each person.
Action: read_slack
Action Input: None
Observation: user,text,ts
Rocky Yu,Dinner will be ready in 20min or so ,2023-07-22 17:32:58
Rocky Yu,Picking up food now ,2023-07-22 17:32:42
Eduardo Reis,"Hey everyone, letting you know that few emails from <http://llama-api.com|llama-api.com> are going to spam for some people, if you didn’t get the invite, please check you spam box.",2023-07-22 16:53:27
Robert Nowell,"Hi folks, anyone want an extra pair of hands to test / improve prompts? Feel free to message",2023-07-22 16:15:22
Ron Bodkin,"Hi all - I wanted to share a link our new open source framework for building with collaborating agents, Council: <https://github.com/chain-ml/council> - I'd love to get feedback and be happy to answer questions if you try it out.",2023-07

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 17789 tokens. Please reduce the length of the messages.